In [2]:
import numpy as np
import pandas as pd
import sys
#sys.path.append("../2021-msia423-hakimi-ben-assignment1")
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
import sklearn
from sklearn.ensemble import RandomForestClassifier

In [3]:
sys.path.append("../../2021-msia423--hakimi---ben--project")

In [4]:
data = pd.read_csv("../../data/data.csv")

In [5]:
data.head()

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,...,2H,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,year
0,906,301,V,NewOrleans,0,10,0,0,10,50.5,...,24.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007
1,906,302,H,Indianapolis,7,3,14,17,41,5.5,...,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007
2,909,405,V,KansasCity,0,0,3,0,3,pk,...,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007
3,909,406,H,HoustonTexans,0,10,7,3,20,38.5,...,pk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007
4,909,407,V,Denver,3,3,6,3,15,3,...,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007


In [5]:
def removePK(rowcount, data):
    ## Turn "pk" games into 0 spread games & convert lines to np.int64
    i = 0
    while i < rowcount:
        if data['Open'][i]=="pk":
            data['Open'][i]=np.int64(0)
        elif type(data['Open'][i])==float:
            data['Open'][i]=np.int64(0)
        else:
            data['Open'][i]=np.int64(float((data['Open'][i])))
        i=i+1
    return data

In [74]:
def oneLine(rowcount, data):
    ## Turn "pk" games into 0 spread games & convert lines to np.int64
    data['Team_2']=""
    data['Final_2']=0
    data['Open_2']=0
    data['ML_2']=0
    ## Move data into one line
    i = 0
    while i < rowcount:
        data['Team_2'][i] = data['Team'][i+1] 
        data['Final_2'][i] = data['Final'][i+1] 
        data['Open_2'][i] = data['Open'][i+1] 
        data['ML_2'][i] = data['ML'][i+1] 
        i = i+2
    return data

In [83]:
def findFavored(rowcount, data):
    ## Determine favored team    
    data['Fav'] = "V"
    i = 0
    while i < rowcount:
        if data['Open'][i] > data['Open_2'][i]:
            data['Fav'][i] = "H"
        i = i+2
    return data

In [89]:
def correctSpread(rowcount, data):
    ## Get correct Spread    
    data['Spread'] = data['Open_2']
    i = 0
    while i < rowcount:
        if data['Fav'][i] == "V":
            data['Spread'][i] = data['Open'][i]
        i = i+2
    return data

In [95]:
def homePick(rowcount, data):
    ## Home team selected as favorite in pick-ems
    i = 0
    while i < rowcount:
        if data['Spread'][i] == 0:
            data['Fav'][i] = "H"
        i = i+2
    return data

In [100]:
def whoCovered(rowcount, data):
    ## Final score diff in terms of home team
    data['Diff'] = data['Final_2']-data['Final']
    
    ## If home team covered, H_cov = 1, if not = 0, if push = 2
    data['H_cov']=0
    i = 0
    while i < rowcount:
        if (data['Fav'][i] == "H") & ((data['Diff'][i]) > (data['Spread'][i])):
            data['H_cov'][i]=1
        elif (data['Fav'][i] == "V") & ((data['Diff'][i])*-1 < (data['Spread'][i])):
            data['H_cov'][i]=1
        elif (data['Fav'][i] == "H") & ((data['Diff'][i]) == (data['Spread'][i])):
            data['H_cov'][i]=2
        elif (data['Fav'][i] == "V") & ((data['Diff'][i])*-1 == (data['Spread'][i])):
            data['H_cov'][i]=2
        i = i+2
    return data

In [108]:
def homeSpread(rowcount, data):
    ## Get spread in terms of home team
    data['homeSpread'] = data['Spread']
    i = 0
    while i < rowcount:
        if data['Fav'][i] == "H":
            data['homeSpread'][i] = data['Spread'][i]*(-1)
        i = i+2
    return data

In [13]:
def tidyUp(data):
    ## Reduce to one line per game
    dataUse = data[data['VH']=='V']
    dataUse.reset_index(drop=True)
    
    ## Get Regular Season Data and Rename Columns
    dataRegSeason = dataUse[['Date','Team','Team_2','Fav','Spread','Diff','H_cov','homeSpread','year']][0:256]
    dataRegSeason.columns = ['Date','V_Team','H_Team','Favorite','Spread','H_diff','Home_Cover','homeSpread','year']
    return dataRegSeason

In [14]:
def seasonSummary(year, data):
    data = data[data['year']==year]
    data = data.reset_index(drop=True)
    rowcount = data.shape[0]
    data = removePK(rowcount, data)
    data = oneLine(rowcount, data)
    data = findFavored(rowcount, data)
    data = correctSpread(rowcount, data)
    data = homePick(rowcount, data)
    data = whoCovered(rowcount, data)
    data = homeSpread(rowcount, data)
    data = tidyUp(data)
    return data


In [15]:
data2007 = seasonSummary(2007, data)
data2008 = seasonSummary(2008, data)
data2009 = seasonSummary(2009, data)
data2010 = seasonSummary(2010, data)
data2011 = seasonSummary(2011, data)
data2012 = seasonSummary(2012, data)
data2013 = seasonSummary(2013, data)
data2014 = seasonSummary(2014, data)
data2015 = seasonSummary(2015, data)
data2016 = seasonSummary(2016, data)
data2017 = seasonSummary(2017, data)
data2018 = seasonSummary(2018, data)
data2019 = seasonSummary(2019, data)
data2020 = seasonSummary(2020, data)
frames = [data2007, data2008, data2009, data2010, data2011, data2012, data2013
         , data2014, data2015, data2016, data2017, data2018, data2019, data2020]

allSeasons = pd.concat(frames)
allSeasons

,Date,V_Team,H_Team,Favorite,Spread,H_diff,Home_Cover,homeSpread,year
0,906,NewOrleans,Indianapolis,H,5,31,1,-5,2007
2,909,KansasCity,HoustonTexans,H,0,17,1,0,2007
4,909,Denver,Buffalo,V,3,-1,1,3,2007
6,909,Pittsburgh,Cleveland,V,4,-27,0,4,2007
8,909,Tennessee,Jacksonville,H,6,-3,0,-6,2007
...,...,...,...,...,...,...,...,...,...
502,103,Miami,Buffalo,H,5,30,1,-5,2020
504,103,LAChargers,KansasCity,H,1,-17,0,-1,2020
506,103,Seattle,SanFrancisco,V,6,-3,1,6,2020
508,103,LasVegas,Denver,H,2,-1,0,-2,2020


In [16]:
allSeasons = allSeasons[allSeasons['Home_Cover']!=2]
allSeasons

,Date,V_Team,H_Team,Favorite,Spread,H_diff,Home_Cover,homeSpread,year
0,906,NewOrleans,Indianapolis,H,5,31,1,-5,2007
2,909,KansasCity,HoustonTexans,H,0,17,1,0,2007
4,909,Denver,Buffalo,V,3,-1,1,3,2007
6,909,Pittsburgh,Cleveland,V,4,-27,0,4,2007
8,909,Tennessee,Jacksonville,H,6,-3,0,-6,2007
...,...,...,...,...,...,...,...,...,...
502,103,Miami,Buffalo,H,5,30,1,-5,2020
504,103,LAChargers,KansasCity,H,1,-17,0,-1,2020
506,103,Seattle,SanFrancisco,V,6,-3,1,6,2020
508,103,LasVegas,Denver,H,2,-1,0,-2,2020


In [17]:
useful = allSeasons[['V_Team', 'H_Team','homeSpread','Home_Cover','year']]
useful['V_Team'] = useful['V_Team'] + "V"
useful.head()

,V_Team,H_Team,homeSpread,Home_Cover,year
0,NewOrleansV,Indianapolis,-5,1,2007
2,KansasCityV,HoustonTexans,0,1,2007
4,DenverV,Buffalo,3,1,2007
6,PittsburghV,Cleveland,4,0,2007
8,TennesseeV,Jacksonville,-6,0,2007


In [18]:
oneHotV = pd.get_dummies(useful['V_Team'])
useful = useful.drop('V_Team',axis = 1)
useful = pd.concat([useful, oneHotV], axis = 1)

In [19]:
oneHotH = pd.get_dummies(useful['H_Team'])
useful = useful.drop('H_Team',axis = 1)
useful = pd.concat([useful, oneHotH], axis = 1)

In [27]:
accs = []
for i in range(1000):
    target = useful['Home_Cover']
    features = useful.drop('Home_Cover',axis = 1)
    features = features.drop('year',axis = 1)
    rf = sklearn.ensemble.RandomForestClassifier(n_estimators=8, max_depth=2)
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
        features, target, test_size=0.25)
    rf.fit(X_train, y_train)
    ypred_proba_test = rf.predict_proba(X_test)[:,1]
    ypred_bin_test = rf.predict(X_test)
    auc = sklearn.metrics.roc_auc_score(y_test, ypred_proba_test)
    confusion = sklearn.metrics.confusion_matrix(y_test, ypred_bin_test)
    accuracy = sklearn.metrics.accuracy_score(y_test, ypred_bin_test)
    classification_report = sklearn.metrics.classification_report(y_test, ypred_bin_test)
    accs.append(accuracy)
    
np.mean(accs)

0.5102421052631579

In [125]:
features

,homeSpread,Home_Cover,ArizonaV,AtlantaV,BaltimoreV,BuffaloBillsV,BuffaloV,CarolinaV,ChicagoV,CincinnatiV,...,Philadelphia,Pittsburgh,SanDiego,SanFrancisco,Seattle,St.Louis,Tampa,TampaBay,Tennessee,Washington
0,-5,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,-6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502,-5,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
504,-1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
506,6,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
508,-2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [109]:
rf = sklearn.ensemble.RandomForestClassifier(n_estimators=10, max_depth=10)

In [110]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    features, target, test_size=0.25)

In [111]:
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, n_estimators=10)

In [112]:
ypred_proba_test = rf.predict_proba(X_test)[:,1]
ypred_bin_test = rf.predict(X_test)

In [113]:
auc = sklearn.metrics.roc_auc_score(y_test, ypred_proba_test)
confusion = sklearn.metrics.confusion_matrix(y_test, ypred_bin_test)
accuracy = sklearn.metrics.accuracy_score(y_test, ypred_bin_test)
classification_report = sklearn.metrics.classification_report(y_test, ypred_bin_test)

In [114]:
accuracy

0.5356725146198831

In [118]:
df_in = pd.DataFrame([[1,"BuffaloBills", "LasVegas"], [0,"SanDiego", "LARams"], [1,"Buffalo", "LAChargers"], [2,"HoustonTexans", "Washington"], [0,"KansasCity", "Houston"]], columns = ["Home_Cover", "H_Team", "V_Team"])
df_in

,Home_Cover,H_Team,V_Team
0,1,BuffaloBills,LasVegas
1,0,SanDiego,LARams
2,1,Buffalo,LAChargers
3,2,HoustonTexans,Washington
4,0,KansasCity,Houston


In [119]:
def fixNames(data, side, homeCol, roadCol):
    """
    Renames columns to account for teams moving/rebranding and human input errors

    Args:
        data: dataframe, required
        side: (str), required, home or away
        homeCol: (str), required, name of column where Home Team name is stored
        roadCol: (str), required, name of column where Road Team name is stored
    
    Returns:
        dataframe
    """
    try:
        numLines = data.shape[0]
        if side == "home":
            i = 0
            while i < numLines:
                if data['H_Team'][i]=="BuffaloBills":
                    data['H_Team'][i]="Buffalo"
                elif data['H_Team'][i]=="LasVegas" or data['H_Team'][i]=="Oakland":
                    data['H_Team'][i]="LVRaiders"
                elif data['H_Team'][i]=="SanDiego":
                    data['H_Team'][i]="LAChargers"
                elif data['H_Team'][i]=="SanDiego":
                    data['H_Team'][i]="LAChargers"
                elif data['H_Team'][i]=="Washingtom":
                    data['H_Team'][i]="Washington"
                elif data['H_Team'][i]=="LosAngeles" or data['H_Team'][i]=="St.Louis":
                    data['H_Team'][i]="LARams"
                elif data['H_Team'][i]=="HoustonTexans":
                    data['H_Team'][i]="Houston"
                elif data['H_Team'][i]=="NewYork":
                    data['H_Team'][i]="NYGiants"
                elif data['H_Team'][i]=="Tampa":
                    data['H_Team'][i]="TampaBay"
                elif data['H_Team'][i]=="Kansas" or data['H_Team'][i]=="KCChiefs":
                    data['H_Team'][i]="KansasCity"
                i = i+1    
        elif side == "away":
            i = 0
            while i < numLines:
                if data['V_Team'][i]=="BuffaloBills":
                    data['V_Team'][i]="Buffalo"
                elif data['V_Team'][i]=="LasVegas" or data['V_Team'][i]=="Oakland":
                    data['V_Team'][i]="LVRaiders"
                elif data['V_Team'][i]=="SanDiego":
                    data['V_Team'][i]="LAChargers"
                elif data['V_Team'][i]=="SanDiego":
                    data['V_Team'][i]="LAChargers"
                elif data['V_Team'][i]=="Washingtom":
                    data['V_Team'][i]="Washington"
                elif data['V_Team'][i]=="LosAngeles" or data['V_Team'][i]=="St.Louis":
                    data['V_Team'][i]="LARams"
                elif data['V_Team'][i]=="HoustonTexans":
                    data['V_Team'][i]="Houston"
                i = i+1    
        return data
    except:
        return "hi"


In [123]:
fixNames(df_in, "away", "H_Team", "V_Team")

,Home_Cover,H_Team,V_Team
0,1,Buffalo,LVRaiders
1,0,LAChargers,LARams
2,1,Buffalo,LAChargers
3,2,Houston,Washington
4,0,KansasCity,Houston


In [124]:
df_test = pd.DataFrame([[1,"Buffalo", "LVRaiders"], [0,"LAChargers", "LARams"], [1,"Buffalo", "LAChargers"], [2,"Houston", "Washington"], [0,"KansasCity", "Houston"]], columns = ["Home_Cover", "H_Team", "V_Team"])
df_test

,Home_Cover,H_Team,V_Team
0,1,Buffalo,LVRaiders
1,0,LAChargers,LARams
2,1,Buffalo,LAChargers
3,2,Houston,Washington
4,0,KansasCity,Houston


In [125]:
df_test.equals(fixNames(df_in, "home", "H_Team", "V_Team"))

True

In [126]:
df_in = pd.DataFrame([[1,"BuffaloBills", "LasVegas"], [0,"SanDiego", "LARams"], [1,"Buffalo", "LAChargers"], [2,"HoustonTexans", "Washington"], [0,"KansasCity", "Houston"]], columns = ["Home_Cover", "H_Team", "V_Team"])


In [142]:
def noPush(data, colName):
    """
    Removes data where outcome is a push

    Args:
        data: dataframe, required
        colName: (str), required, name of column where cover information is stored
    
    Returns:
        dataframe
    """
    try:
        data = data[data['Home_Cover']!=2]
        data = data.reset_index(drop=True)
        return data
    except:
        return "hi"

In [143]:
noPush(df_in, "Home_Cover")

,Home_Cover,H_Team,V_Team
0,1,BuffaloBills,LasVegas
1,0,SanDiego,LARams
2,1,Buffalo,LAChargers
3,0,KansasCity,Houston


In [139]:
df_in[df_in["Home_Cover"]!=2]

,Home_Cover,H_Team,V_Team
0,1,BuffaloBills,LasVegas
1,0,SanDiego,LARams
2,1,Buffalo,LAChargers
4,0,KansasCity,Houston


In [ ]:
df_in = pd.DataFrame([[1,"BuffaloBills", "LasVegas"], [0,"SanDiego", "LARams"], [1,"Buffalo", "LAChargers"], [0,"KansasCity", "Houston"]], columns = ["Home_Cover", "H_Team", "V_Team"])
